In [2]:
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import models, evaluation
import os
import pandas as pd
import warnings 
warnings.filterwarnings("ignore")

In [52]:
pretrain_model_path = "/Users/zyl/Desktop/LLM/pretrain_models/all-MiniLM-L6-v2"
# pretrain_model_path = "/Users/zyl/Desktop/LLM/pretrain_models/quora-distilbert-multilingual"
model = SentenceTransformer(pretrain_model_path)

In [45]:
emb1 = model.encode("CPU使用率大于90%", convert_to_tensor=True)
emb2 = model.encode("CPU使用率小于90%", convert_to_tensor=True)
# print(emb1)
# print(emb2)


In [46]:
cos_sim = util.cos_sim(emb1, emb2)
print("Cosine Sim is :", cos_sim)

Cosine Sim is : tensor([[0.9777]])


In [47]:
emb1 = model.encode("CPU使用率大于90%", convert_to_tensor=True)
emb2 = model.encode("CPU使用率大于80%", convert_to_tensor=True)
# print(len(emb1))
# print(emb2)

In [48]:
cos_sim = util.cos_sim(emb1, emb2)
print("Cosine Sim is :", cos_sim)

Cosine Sim is : tensor([[0.9629]])


In [49]:
emb1 = model.encode("CPU使用率大于90%", convert_to_tensor=True)
emb2 = model.encode("CPU使用率大于70%", convert_to_tensor=True)
cos_sim = util.cos_sim(emb1, emb2)
print("Cosine Sim is :", cos_sim)

Cosine Sim is : tensor([[0.9539]])


In [51]:
emb1 = model.encode("CPU使用率大于90%", convert_to_tensor=True)
emb2 = model.encode("CPU使用率大于60%", convert_to_tensor=True)
cos_sim = util.cos_sim(emb1, emb2)
print("Cosine Sim is :", cos_sim)

Cosine Sim is : tensor([[0.9386]])


In [43]:
emb1 = model.encode("CPU使用率大于90%", convert_to_tensor=True)
emb2 = model.encode("CPU使用率大于40%", convert_to_tensor=True)
cos_sim = util.cos_sim(emb1, emb2)
print("Cosine Sim is :", cos_sim)

Cosine Sim is : tensor([[0.9514]])


In [57]:
sentences = [
    "CPU 使用率超过95%",
    "华为交换机CPU平均使用率超过 70%",
    "华为交换机CPU使用率超过70%",
    "CPU使用率过高，超过75%",
    "CPU iowait 使用率超过 50%",
    "CPU使用率低于10%", 
    
    "磁盘分区 / 使用率超过 90%",
    "分区:/opt使用率超过80%",
    "磁盘 /data 使用率超过80%",
    "C:盘使用率超过85%",
    "磁盘使用率超过 80%",
    "磁盘 / 使用率超过90%",
    "磁盘 /home/agriculture/mcms 使用率超过80%"

    "华三交换机内存使用率过高",
    "华三交换机内存未正常运行",
    "华三交换机电源未正常运行"
    "华三交换机电源未正常运行",
    "华为交换机CPU使用率超过70%",
    # "redis连接少于10", 
    # "10.2.23.1/10.2.23.1/cpu/暂未匹配,指标检测随机丢失率告警"
]

embeddings = model.encode(sentences)
cos_sim = util.cos_sim(embeddings, embeddings)    # 每两个都计算一次
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

df = pd.DataFrame()
l1,l2,l3 = [],[],[]
for score, i, j in all_sentence_combinations[:]:
    l1.append(sentences[i])
    l2.append(sentences[j])
    l3.append(cos_sim[i][j].item())
    # print("{} \t {} \t {:.4f}".format(sentences[i], sentences[j], cos_sim[i][j]))
df['sentence1'] = l1
df['sentence2'] = l2
df['cos_sim'] = l3
display(df.head(20))

,sentence1,sentence2,cos_sim
0,华为交换机CPU使用率超过70%,华为交换机CPU使用率超过70%,1.000000
1,CPU 使用率超过95%,CPU使用率低于10%,0.926380
2,磁盘分区 / 使用率超过 90%,分区:/opt使用率超过80%,0.925554
3,华为交换机CPU使用率超过70%,CPU使用率低于10%,0.887013
4,CPU使用率低于10%,华为交换机CPU使用率超过70%,0.887013
5,CPU 使用率超过95%,华为交换机CPU使用率超过70%,0.879013
6,CPU 使用率超过95%,华为交换机CPU使用率超过70%,0.879013
7,华三交换机内存未正常运行,华三交换机电源未正常运行华三交换机电源未正常运行,0.870063
8,磁盘使用率超过 80%,磁盘 / 使用率超过90%,0.854156
9,华为交换机CPU平均使用率超过 70%,华为交换机CPU使用率超过70%,0.849644


In [ ]:
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

In [2]:
from datetime import datetime, timezone, timedelta

def reverse_num_to_time(t):
    timestamp = t / 1000.0
    dt_utc = datetime.utcfromtimestamp(timestamp).replace(tzinfo=timezone.utc)
    dt_local = dt_utc.astimezone(timezone(timedelta(hours=8)))
    formatted_time = dt_local.strftime('%Y-%m-%dT%H:%M:%S.%f%z')
    return formatted_time

t = 1676448543000
print(reverse_num_to_time(t))

2023-02-15T16:09:03.000000+0800
